In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Prime Librerie da importare**

In [ ]:
import cv2
import numpy as np
import dlib
import math
import os
from pathlib import Path

# Path principali delle cartelle in cui sono contenuti i file

In [ ]:
path_principale = "/content/drive/Shareddrives/Progetti FVAB 22 23 - VSR & BABELE/Gruppi/Gruppo 22/Progetto BABELE/"
path_train = "Train/"
path_test = "Test/"
path_validation = "Validation/"

# **Salvataggio lista di librerie e versioni installate**

In [ ]:
!pip freeze > "/content/drive/Shareddrives/Progetti FVAB 22 23 - VSR & BABELE/Gruppi/Gruppo 22/Progetto BABELE/requirements_estrazioneFeatures.txt"

# **Creazioni directory importanti per il progetto**

In [ ]:
# Funzione per creare una directory se non esiste già
def create_directory_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Creazione delle directory principali
create_directory_if_not_exists(path_principale + "CSV_DATASET")
create_directory_if_not_exists(path_principale + "CSV_DATASET_MERGED")
create_directory_if_not_exists(path_principale + "CSV_DATASET_SVD")
create_directory_if_not_exists(path_principale + "CSV_PNN_RESULT")
create_directory_if_not_exists(path_principale + "CSV_PNN_RESULT_MERGED")
create_directory_if_not_exists(path_principale + "CSV_PNN_RESULT_SVD")
create_directory_if_not_exists(path_principale + "CSV_DATASET_CH-SIMS_CMU-MOSEI")
create_directory_if_not_exists(path_principale + "FILE_PATH")

# Creazione delle sotto-directory per train, test e validation
create_directory_if_not_exists(path_principale + "CSV_DATASET/" + path_train)
create_directory_if_not_exists(path_principale + "CSV_DATASET/" + path_test)
create_directory_if_not_exists(path_principale + "CSV_DATASET/" + path_validation)

create_directory_if_not_exists(path_principale + "CSV_DATASET_MERGED/" + path_train)
create_directory_if_not_exists(path_principale + "CSV_DATASET_MERGED/" + path_test)
create_directory_if_not_exists(path_principale + "CSV_DATASET_MERGED/" + path_validation)

create_directory_if_not_exists(path_principale + "CSV_DATASET_SVD/" + path_train)
create_directory_if_not_exists(path_principale + "CSV_DATASET_SVD/" + path_test)
create_directory_if_not_exists(path_principale + "CSV_DATASET_SVD/" + path_validation)

create_directory_if_not_exists(path_principale + "CSV_PNN_RESULT/" + path_train)
create_directory_if_not_exists(path_principale + "CSV_PNN_RESULT/" + path_test)
create_directory_if_not_exists(path_principale + "CSV_PNN_RESULT/" + path_validation)

create_directory_if_not_exists(path_principale + "CSV_PNN_RESULT_MERGED/" + path_train)
create_directory_if_not_exists(path_principale + "CSV_PNN_RESULT_MERGED/" + path_test)
create_directory_if_not_exists(path_principale + "CSV_PNN_RESULT_MERGED/" + path_validation)

create_directory_if_not_exists(path_principale + "CSV_PNN_RESULT_SVD/" + path_train)
create_directory_if_not_exists(path_principale + "CSV_PNN_RESULT_SVD/" + path_test)
create_directory_if_not_exists(path_principale + "CSV_PNN_RESULT_SVD/" + path_validation)

create_directory_if_not_exists(path_principale + "CSV_DATASET_CH-SIMS_CMU-MOSEI/" + path_train)
create_directory_if_not_exists(path_principale + "CSV_DATASET_CH-SIMS_CMU-MOSEI/" + path_test)

create_directory_if_not_exists(path_principale + "CSV_DATASET_OPTICAL_FLOW/" + path_train)
create_directory_if_not_exists(path_principale + "CSV_DATASET_OPTICAL_FLOW/" + path_test)
create_directory_if_not_exists(path_principale + "CSV_DATASET_OPTICAL_FLOW/" + path_validation)

create_directory_if_not_exists(path_principale + "CSV_DATASET_OPTICAL_FLOW/" + path_train + "TWO_FRAME/")
create_directory_if_not_exists(path_principale + "CSV_DATASET_OPTICAL_FLOW/" + path_test + "TWO_FRAME/")
create_directory_if_not_exists(path_principale + "CSV_DATASET_OPTICAL_FLOW/" + path_validation + "TWO_FRAME/")

create_directory_if_not_exists(path_principale + "CSV_DATASET_OPTICAL_FLOW/" + path_train + "SEQUENCE/")
create_directory_if_not_exists(path_principale + "CSV_DATASET_OPTICAL_FLOW/" + path_test + "SEQUENCE/")
create_directory_if_not_exists(path_principale + "CSV_DATASET_OPTICAL_FLOW/" + path_validation + "SEQUENCE/")

create_directory_if_not_exists(path_principale + "FILE_PATH/" + path_train)
create_directory_if_not_exists(path_principale + "FILE_PATH/" + path_test)
create_directory_if_not_exists(path_principale + "FILE_PATH/" + path_validation)

# **Configurazione mediapipe**

## Installazione mediapipe

In [ ]:
!pip install mediapipe

## Coordinate dei landmark delle labbra di mediapipe

In [ ]:
FACEMESH_LIPS = [(61,61),(37,37),(0, 0)(267,267),(306,306),(314,314),(17, 17),(84,84)]

# **Funzioni per il calcolo dei landmark**

## **Funzione che calcola la distanza tra due landmark sfruttando la distanza cityblock**

In [ ]:
from scipy.spatial import distance

def calcola_distanza_landmark(selected_points,  dist="cityblock"):
  if any(math.isnan(val) for coord in selected_points for val in coord):
        return math.nan
  # Inizializzazione della variabile per la lunghezza totale
  total_length = 0

  # Iterazione su tutti i landmark selezionati in ordine
  for i in range(len(selected_points)-1):
      # Estrazione delle coordinate del landmark corrente
      p1= selected_points[i]
      
      # Estrazione delle coordinate del landmark successivo
      p2 = selected_points[i+1]
      # cityblock, cosine, chebyshev, euclidean
      # cityblock, cosine, chebyshev, euclidean
      if dist == "cityblock":
        distanza = distance.cityblock(p1, p2)
      elif dist == "cosine":
        distanza = distance.cosine(p1, p2)
      elif dist == "chebyshev":
        distanza = distance.chebyshev(p1, p2)
      elif dist == "euclidean":
        distanza = distance.euclidean(p1, p2)
      else:
        raise ValueError("Valore di dist non riconosciuto")
      
      
      # Aggiornamento della lunghezza totale
      total_length += distanza
      #total_length = round(total_length,2)
  
  return total_length

## **Funzione che estrae la larghezza delle labbra in pixel**

In [ ]:
# larghezza delle labbra su tre punti specifici
def estrazione_feature_larghezza(landmarks_points, distanza="cityblock"):

  zona_superiore = [landmarks_points[37],  landmarks_points[267]]

  zona_centrale =  [landmarks_points[61],  landmarks_points[306] ]

  zona_inferiore = [landmarks_points[84],  landmarks_points[314]]

  w1 = calcola_distanza_landmark(zona_superiore, distanza)
  w2 = calcola_distanza_landmark(zona_centrale, distanza)
  w3 = calcola_distanza_landmark(zona_inferiore, distanza)

  return {"37-267":w1, "61-306": w2,"84-314": w3}

## **Funzione che estrae l'altezza delle labbra in pixel**

In [ ]:
# Altezza delle labbra utilizzando i tubercoli
def estrazione_feature_altezza(landmarks_points, distanza="cityblock"):

  tubercolo_sinistro_superiore = [landmarks_points[37],  landmarks_points[84]]

  tubercolo_centrale_superiore = [landmarks_points[0],  landmarks_points[17]]

  tubercolo_destro_superiore = [landmarks_points[267],  landmarks_points[314]]

  h1 = calcola_distanza_landmark(tubercolo_sinistro_superiore,  distanza)
  h2 = calcola_distanza_landmark(tubercolo_centrale_superiore,  distanza)
  h3 = calcola_distanza_landmark(tubercolo_destro_superiore, distanza)


  return {"37-84":h1,"0-17": h2,"267-314":h3}

## **Funzione utilizzata per settare i valori dei landmark a (0,0) in caso che il volto non venga rilevato**

In [ ]:
# funzione che controlla se un landmark è stato trovato, se non è stato trovato setta il suo valore a NaN
def key_exists(landmarks_points):
  temp_value = [math.nan, math.nan]
  if 37 not in landmarks_points:
    landmarks_points[37] = temp_value
  if 267 not in landmarks_points:
    landmarks_points[267] = temp_value
  if 61 not in landmarks_points:
    landmarks_points[61] = temp_value
  if 306 not in landmarks_points:
    landmarks_points[306] = temp_value
  if 84 not in landmarks_points:
    landmarks_points[84] = temp_value
  if 314 not in landmarks_points:
    landmarks_points[314] = temp_value
  if 0 not in landmarks_points:
    landmarks_points[0] = temp_value
  if 17 not in landmarks_points:
    landmarks_points[17] = temp_value

## **Funzione che genera un dizionario contenente le feature delle labbra**

In [ ]:
def generaFeature(landmarks_points,distanza="cityblock"):
  key_exists(landmarks_points)
  feature2 = estrazione_feature_larghezza(landmarks_points, distanza)
  feature3 = estrazione_feature_altezza(landmarks_points, distanza)

  return dict(feature2,  **feature3)

# **Funzione per toglierre i NaN**


In [ ]:
import pandas as pd


# Funzione che riempie i valori NaN con la media mobile calcolata su una finestra mobile di lunghezza n_frame
# Il tutto a condizione che ci siano almeno 2 valori non NaN nella finestra (min_periods)
def fill_nan(row, n_frame):
    return row.fillna(row.rolling(min_periods=2, window=n_frame).mean())

# Funzione per sostituire i valori NaN all'inizio del file
def remove_initial_nan(df):
  # Sostituzione dei valori NaN nelle colonne intermedie con la media dei successivi
  df_filled = df.copy()
  columns_to_fill = df.columns[1:-1]  # Seleziona tutte le colonne tranne la prima e l'ultima

  for column in columns_to_fill:
      df_filled[column].fillna(df_filled[column].fillna(method='ffill').shift(-1).mean(), inplace=True)
    
  return df_filled

# Funzione per rimuovere i valori NaN
def remove_nan(df,num_frames_per_second):
  df = remove_initial_nan(df)
  df = df.apply(fill_nan, axis=1, n_frame=num_frames_per_second) #Caso di NaN in altre posizioni

  return df

# **Generazione dei file in cui saranno contenute le path dei video**

## Creazione file path video Babele

La nomenclatura proposta è stata:
*Language_gender_age_id-unique_framerate*

In merito a questo il csv in output sarà così formato:
*   Nome del file: *Language_gender_age_id-unique_framerate_numeroframe*
*   Features estratte: distanza labbra , ecc
*   Target: lingua associata al frame con le feature estratte


 

In [ ]:
import os
import glob

# cartella contenente la path del dataset
dataset_path = "/content/drive/Shareddrives/Progetti FVAB 22 23 - VSR & BABELE/Datasets/Dataset_2_BABELE/BABELE_Dataset/Dataset_split_subject_independent/Full_video_10_seconds_division_train_test_val"

# cartella in cui salvare i file
folder_save = "/content/drive/Shareddrives/Progetti FVAB 22 23 - VSR & BABELE/Gruppi/Gruppo 22/Progetto BABELE/FILE_PATH"

# Loop over all subfolders in folder_read
for subfolder in os.listdir(dataset_path):
    # Create a file with the subfolder's name in folder_save
    file_path = os.path.join(folder_save, subfolder + "_babele.txt")
    
    # Open the file for writing
    with open(file_path, "w") as f:
        # Get a list of all files in the subfolder
        files = glob.glob(os.path.join(dataset_path, subfolder, "*"))
        
        # Write each file's path to the file
        for file in files:
            f.write(file + "\n")


# **Preprocessing dataset lingue vs Mondo**

*   Estrazione dei landmark dai frame video
*   Calcolo distanze fra i landmark
*   Creazione di un file csv per ogni file di testo che contiene le path dei video




## Lista contenente le path dei file dei video

In [ ]:
files_path = ["/content/drive/Shareddrives/Progetti FVAB 22 23 - VSR & BABELE/Gruppi/Gruppo 22/Progetto BABELE/FILE_PATH/Test_babele.txt",
              "/content/drive/Shareddrives/Progetti FVAB 22 23 - VSR & BABELE/Gruppi/Gruppo 22/Progetto BABELE/FILE_PATH/Validation_babele.txt",
              "/content/drive/Shareddrives/Progetti FVAB 22 23 - VSR & BABELE/Gruppi/Gruppo 22/Progetto BABELE/FILE_PATH/Train_babele.txt"
              ]

## **Creazione dataset binario Tedesco vs Mondo**

### Bilanciamento dataset tra Tedesco e altre lingue

In [ ]:
# Funzione che divide il dataset in due sottoinsiemi in base alla classe di appartenenza
def bilanciamento_dataset(df):
  class_0 = df[df['target'] == 0]
  class_1 = df[df['target'] == 1]

  # determiniamo qunati elementi prendere dalla classe di maggioranza basandoci sul numero di elementi disponibili
  # della classe di minoranza
  num_samples = len(class_1)

  # seleziona un numero casuale di campioni dalla classe di maggioranza
  # random_state è usato come seed di partenza per la randomizzazione
  class_0_balanced = class_0.sample(n=num_samples, random_state=42)

  # unisce i due sottoinsiemi bilanciati
  balanced_dataset = pd.concat([class_0_balanced, class_1])

  # mischia il dataset in modo casuale
  balanced_dataset = balanced_dataset.sample(frac=1, random_state=42)

  # stampa il conteggio delle classi nel dataset bilanciato
  print(balanced_dataset['target'].value_counts())

  return balanced_dataset

### Creazione dataset tra Tedesco e altre lingue

In [ ]:
import random
import cv2
import mediapipe as mp
import pandas as pd
import os
from tqdm import tqdm

fps = 24
num_frames = 240
num_frames_per_second = 6
num_seconds = num_frames // fps

random_indices = []
for i in range(num_seconds):
    start_index = i * fps
    end_index = start_index + fps
    indices = list(range(start_index, end_index))
    random_indices.extend(random.sample(indices, num_frames_per_second))

random_indices.sort()

frames_to_process = [i for i in range(num_frames) if i in random_indices]

print(frames_to_process)

# Caricamento del modello di rilevamento dei landmarks del volto
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

# Inizializza il rilevatore dei landmark del viso
face_mesh = mp_face_mesh.FaceMesh()
distanza="cityblock"

for path_file in files_path:

  with open(path_file, mode='r') as file:

    video_data = []  # Lista di tutti i dati dei video
    
    linee = file.readlines()

    csv_filename = os.path.basename(path_file)
    csv_filename_without_extension = csv_filename.split(".")[0] #per la creazione del dataset voglio un nome senza .txt
    estensione = "_cityblock_tedescoVsMondo.csv"
    
    if "Validation" in path_file:
      csv_path = path_principale+"CSV_DATASET/"+path_validation + csv_filename_without_extension + estensione
    elif "Train" in path_file:
      csv_path = path_principale+"CSV_DATASET/"+path_train + csv_filename_without_extension + estensione
    elif "Test" in path_file:
       csv_path = path_principale+"CSV_DATASET/"+path_test + csv_filename_without_extension + estensione
    else:
      print("Qualcosa è andato storto!")
      break

    for video_path in tqdm(linee, desc="Elaborazione video " + csv_filename_without_extension, unit="video"):
        video_path = video_path.strip() # Rimuovi il carattere di a capo dalla stringa

        # Apre il video in input
        cap = cv2.VideoCapture(str(video_path))

        filename = os.path.basename(video_path) #Nome del file video
        filename_without_extension = filename.split(".")[0] #per la creazione del dataset voglio un nome senza .avi
        components = filename.split("_")[:-1] #splitto l'array in un altro per "_" e elimino ".avi"
       
        target = int(components[0]) #Estrazione classe (lingua)

        if target == 3:
            target = 1
        else:
            target = 0

        # Inizializza la lista dei dati del video
        video_data_video = []

        # Loop attraverso tutti i frame del video
        frame_count = 0
        frame_data = {}
        #nome del frame
        frame_data['video-frame'] = filename_without_extension # Alla fine collego al nome del file il frame analizzato
        colonna = 0

       
        while cap.isOpened() and frame_count < 240:
            

            ret, frame = cap.read()
            if not ret:
                break

            if frame_count not in frames_to_process: #l'indice del frame non si trova in quelli random
              frame_count += 1
              continue
          
            # Converte l'immagine in RGB
            frame_RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            if frame_RGB is None:
              print("Frame errato, la funzione cv2.COLOR_BGR2RGB non è stata in grado di convertire l'immagine in RGB")
              frame_count += 1
              continue
            else:
              results = face_mesh.process(frame_RGB)

              if results is None:
                print("Qualcosa è andato storto!")
                frame_count += 1
                continue
              else:
                # Memorizza i punti dei landmarks delle labbra
                landmarks_points = {}
                if results.multi_face_landmarks:
                    face_landmarks = results.multi_face_landmarks[0]
                    for connection in FACEMESH_LIPS:
                        landmark_1 = face_landmarks.landmark[connection[0]]
                        n = connection[0]

                        # Salva i dati del landmark
                        landmarks_points[n] = [landmark_1.x, landmark_1.y]

                
                feature = generaFeature(landmarks_points, distanza)

                video_data_video
                
                # Crea il dizionario con i dati del frame
                frame_data['37-267_'+str(colonna)] = feature['37-267']
                frame_data['61-306_'+str(colonna)]= feature['61-306']
                frame_data['84-314_'+str(colonna)]= feature['84-314']
                frame_data['37-84_'+str(colonna)]= feature['37-84']
                frame_data['0-17_'+str(colonna)]= feature['0-17']
                frame_data['267-314_'+str(colonna)]= feature['267-314']

                frame_count += 1
                colonna += 1 

        frame_data['target']=  target
        
        # Aggiungi il dizionario alla lista dei dati del video
        video_data_video.append(frame_data)

        # Aggiungi i dati del video alla lista di tutti i dati
        video_data.extend(video_data_video)

        # Chiudi il video
        cap.release()

    # Crea il dataframe dai dati di tutti i video e visualizzalo
    df = pd.DataFrame(video_data)

    ## Check per valori NaN
    df = remove_nan(df,(num_frames_per_second*10))

    dataset_balanced = bilanciamento_dataset(df) 
    # Visualizza il DataFrame
    display(dataset_balanced , num_frames_per_second)

    print(100*"-")
    print("Generazione del file csv ", csv_filename_without_extension, estensione)
    print(100*"-")
    dataset_balanced.to_csv(csv_path, index=False)

    file.close()

# Chiudi il rilevatore dei landmark del viso
face_mesh.close()

## Creazione dataset binarie tra le lingue

In [ ]:
def genera_dataset_binario(df,folder,file_name,lan_target):
  # Crea una copia del dataset
  df_copia = df.copy()

  # Estrai il valore di "Language" dal video-frame
  df_copia["Language"] = df_copia["video-frame"].str.split("_").str[0]

  # Modifica la colonna "target" in base al valore della colonna "language"
  df["target"] = df_copia["Language"].apply(lambda x: 1 if x == lan_target else 0)

  #bilanciamento del dataset
  dataset_balanced = bilanciamento_dataset(df) 

  # Salva la copia del dataset modificato su un nuovo file CSV
  dataset_balanced.to_csv(path_principale+"CSV_DATASET/"+folder+file_name+".csv", index=False)  # Assumi che tu voglia salvare il nuovo file come "nome_file_modificato.csv"

  print("Dataset " + file_name+".csv creato correttamente" )

## Generazione effettiva dei dataset

In [ ]:
import pandas as pd

#Recupero dei vari csv
df_train = pd.read_csv(path_principale+'CSV_DATASET/'+path_train+'Train_babele_cityblock_60FPS.csv')
df_test = pd.read_csv(path_principale+'CSV_DATASET/'+path_test+'Test_babele_cityblock_60FPS.csv')
df_validation = pd.read_csv(path_principale+'CSV_DATASET/'+path_validation+'Validation_babele_cityblock_60FPS.csv')


lingue = {
    1: "italiano",
    2: "inglese",
    3: "tedesco",
    4: "spagnolo",
    5: "olandese",
    6: "russo",
    7: "giapponese",
    8: "francese"
}

for key in lingue:
  if key !=3: # dataset per il tedesco già creato
    genera_dataset_binario(df_train, path_train, "Train_babele_cityblock_"+lingue[key]+"VsMondo",str(key))
    genera_dataset_binario(df_test, path_test, "Test_babele_cityblock_"+lingue[key]+"VsMondo",str(key))
    genera_dataset_binario(df_validation, path_validation, "Validation_babele_cityblock_"+lingue[key]+"VsMondo",str(key))